In [1]:
%load_ext autoreload
%autoreload 2

from api_tricount import *


In [83]:
%load_ext autoreload
%autoreload 2
from api_portfolio import build_data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import streamlit as st
import gspread as gs 
import pandas as pd
import numpy as np
import yfinance as yf

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
ss = gc.open_by_key(st.secrets['portfolio'].spreadsheet_key)
dicts = pd.DataFrame(ss.worksheet('Dict').get_all_records())
operation = pd.DataFrame(ss.worksheet('Operations').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
greenbull = pd.DataFrame(ss.worksheet('GREENBULL').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
assets = dicts.set_index('Asset')
print(assets, '\n')

market = yf.download(' '.join([*assets['Forex'], *assets['Market'][:-1]]), start='2021-04-28')['Close']
print(market.columns)
market = pd.concat([market, greenbull], axis=1).ffill().bfill()
# market = pd.concat([market], keys=['Market'], axis=1)
# market = pd.concat([market], keys=['Cotation'], axis=1)

operation: pd.DataFrame = operation
assets: pd.DataFrame = assets
market: pd.DataFrame = market

df = pd.DataFrame(index=market.index, columns=pd.MultiIndex(levels=[[],[],[],[]], codes=[[],[],[],[]]))#, names=['portfolio', 'class', 'asset', 'metric']))
df.head()

              Market Currency     Forex           Class  PriceFmt AmountFmt  \
Asset                                                                         
EUR         EUREUR=X           EUREUR=X         Deposit                       
€           EUREUR=X      EUR  EUREUR=X            Cash  {:.0f} €    {:.0f}   
$              EUR=X      EUR  EUREUR=X            Cash  {:.4f} €    {:.0f}   
USDT           EUR=X     EUR   EUREUR=X            Cash  {:.4f} €    {:.0f}   
ESE           ESE.PA        €  EUREUR=X        Equities  {:.2f} €    {:.0f}   
PUST                        €  EUREUR=X        Equities  {:.2f} €    {:.0f}   
RS2K                        €  EUREUR=X        Equities  {:.2f} €    {:.0f}   
OBLI         OBLI.PA        €  EUREUR=X    Fixed Income  {:.2f} €    {:.0f}   
GOLD         GOLD.PA        €  EUREUR=X     Commodities  {:.2f} €    {:.0f}   
BTC          BTC-USD     USDT  EURUSD=X  Cryptocurrency  {:.0f} $    {:.4f}   
ETH                      USDT  EURUSD=X  Cryptocurre

Date
2021-04-01
2021-04-28
2021-04-29
2021-04-30
2021-05-01


In [35]:

for portfolio in np.unique(operation['Portfolio']):
    print(portfolio)
    dfp: pd.DataFrame = operation[operation['Portfolio'] == portfolio]

    print(f"\tAssets")
    for asset in np.unique(dfp['Asset']):
        classs = assets.loc[asset]['Class']
        print(f"\t\t{classs}:\t{asset}")

        dfa: pd.DataFrame = dfp[dfp['Asset'] == asset]
        dfa = dfa.groupby('Date').agg({'Quantity': 'sum', 'Operation': 'sum'}).reindex(df.index).fillna(0)

        df[portfolio, classs, asset, 'Quantity'] = dfa['Quantity']
        df[portfolio, classs, asset, 'Operation'] = dfa['Operation']
        df[portfolio, classs, asset, 'Cotation'] = market[assets.loc[asset]['Market']]
    
    print(f"\tCurrency")
    for currency in np.unique(dfp['Currency']):
        classs = assets.loc[currency]['Class']
        print(f"\t\t{classs}:\t{currency}")

        dfc: pd.DataFrame = dfp[dfp['Currency'] == currency]
        dfc = dfc.groupby('Date').agg({'Quantity': 'sum', 'Operation': 'sum'}).reindex(df.index).fillna(0)
        
        if (portfolio, classs, currency, 'Quantity') not in df.columns:
            df[portfolio, classs, currency, 'Quantity'] = 0
        if (portfolio, classs, currency, 'Operation') not in df.columns:
            df[portfolio, classs, currency, 'Operation'] = 0
        if (portfolio, classs, currency, 'Cotation') not in df.columns:
            df[portfolio, classs, currency, 'Cotation'] = market[assets.loc[currency]['Market']]

        df[portfolio, classs, currency, 'Quantity'] -= dfc['Operation']
        df[portfolio, classs, currency, 'Operation'] -= dfc['Operation'] * df[portfolio, classs, currency, 'Cotation']


    print(f"\tBuild datatable ...")
    for asset in np.unique([*dfp['Asset'], *dfp['Currency']]):
        classs = assets.loc[asset]['Class']
        print(f"\t\t{asset}")

        position, invested = pd.Series(dtype=float), pd.Series(dtype=float)
        for idx in df.index:
            position[idx] = np.sum(df.loc[:idx][portfolio, classs, asset, 'Quantity'])
            invested[idx] = np.sum(df.loc[:idx][portfolio, classs, asset, 'Operation'])

        df = pd.concat([df, pd.DataFrame({
                (portfolio, classs, asset, 'Position'): position,
                (portfolio, classs, asset, 'Invested'): invested,
            })], axis=1)

        df[portfolio, classs, asset, 'PRU'] = df[portfolio, classs, asset, 'Invested'] / df[portfolio, classs, asset, 'Position']
        df[portfolio, classs, asset, 'Value'] = df[portfolio, classs, asset, 'Position'] * df[portfolio, classs, asset, 'Cotation']
        df[portfolio, classs, asset, 'PnL'] = df[portfolio, classs, asset, 'Value'] - df[portfolio, classs, asset, 'Invested']
            
        toEUR = market[assets.loc[asset]['Forex']]
        df[portfolio, classs, asset, 'InvestedEUR'] = df[portfolio, classs, asset, 'Invested'] / toEUR
        df[portfolio, classs, asset, 'ValueEUR'] = df[portfolio, classs, asset, 'Value'] / toEUR
        df[portfolio, classs, asset, 'PnLEUR'] = df[portfolio, classs, asset, 'PnL'] / toEUR


    print(f"\tConcate class data ...")
    for classs in np.unique(df[portfolio].columns.get_level_values(0)):
        print(f"\t\t{classs}")
        df[portfolio, classs, 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice[portfolio, classs, :, 'InvestedEUR']].sum(axis=1)
        df[portfolio, classs, 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice[portfolio, classs, :, 'ValueEUR']].sum(axis=1)
        df[portfolio, classs, 'All', 'PnLEUR'] = df.loc[:, pd.IndexSlice[portfolio, classs, :, 'PnLEUR']].sum(axis=1)

    df[portfolio, 'All', 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice[portfolio, :, 'All', 'InvestedEUR']].sum(axis=1)
    df[portfolio, 'All', 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice[portfolio, :, 'All', 'ValueEUR']].sum(axis=1)
    df[portfolio, 'All', 'All', 'PnLEUR'] = df.loc[:, pd.IndexSlice[portfolio, :, 'All', 'PnLEUR']].sum(axis=1)

for classs in np.unique(df.columns.get_level_values(1)):
    df['All', classs, 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice[:, classs, 'All', 'InvestedEUR']].sum(axis=1)
    df['All', classs, 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice[:, classs, 'All', 'ValueEUR']].sum(axis=1)
    df['All', classs, 'All', 'PnLEUR'] = df.loc[:, pd.IndexSlice[:, classs, 'All', 'PnLEUR']].sum(axis=1)

df['All', 'All', 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice['All', :, 'All', 'InvestedEUR']].sum(axis=1)
df['All', 'All', 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice['All', :, 'All', 'ValueEUR']].sum(axis=1)
df['All', 'All', 'All', 'PnLEUR'] = df.loc[:, pd.IndexSlice['All', :, 'All', 'PnLEUR']].sum(axis=1)


for column in df.columns:
    print(column)
# dates = ["2021-04-30", "2021-04-30", "2021-07-30", "2021-07-30", "2021-08-10", "2021-08-10", "2021-10-01", "2021-10-01", "2021-11-01", "2021-11-01", "2021-12-01", "2021-12-01", "2021-12-31", "2022-02-01", "2022-02-01", "2022-03-24", "2022-03-24", "2022-04-01", "2022-04-01", "2022-04-01", "2022-04-05", "2022-04-05", "2022-04-13", "2022-04-13", "2022-05-02", "2022-05-02", "2022-04-19", "2022-04-19", "2022-06-01", "2022-06-01", "2022-06-01", "2022-06-01", "2022-07-01", "2022-07-04", "2022-07-05", "2022-07-05", "2022-08-05", "2022-08-05", "2022-08-25", "2022-10-12", "2022-10-12", "2022-10-12", "2023-02-10"]
dates = ["2021-04-30","2021-07-30","2021-08-10","2021-10-01","2021-11-01","2021-12-01","2021-12-31","2022-02-01","2022-03-24","2022-04-01","2022-04-05","2022-04-13","2022-05-02","2022-04-19","2022-06-01","2022-07-01","2022-07-04","2022-07-05","2022-08-05","2022-08-25","2022-10-12","2023-02-10"]
# df.loc[dates]
# df.head()
df.loc[dates]   #, pd.IndexSlice[['Quantity', 'Operation', 'Total', 'Invested'], 'ZEN', ['BTC', 'USDT', 'EUR']]]


DMA
	Assets
		Cash:	€
	Currency
		Deposit:	EUR
	Build datatable ...
		EUR


ValueError: Columns must be same length as key

In [21]:
np.unique(df['ZEN'].columns.get_level_values(0))

array(['Cash', 'Commodities', 'Cryptocurrency', 'Deposit', 'Equities',
       'Fixed Income'], dtype=object)